# Archery Accuracy Gamma Prior in `RevBayes`

## Background

This Jupyter notebook is intended to demonstrate how to set up a model in RevBayes. The archery example is based on [Paul Lewis](http://phylogeny.uconn.edu/)' lecutres from the [Workshop on Molecular Evolution](https://molevol.mbl.edu/). His [Bayesian lecture slides](http://hydrodictyon.eeb.uconn.edu/people/plewis/downloads/wh2017/Bayesian-Lewis-23July2017.pdf) use this archery to set up the concept of the prior and the data. [Tracy Heath](http://phyloworks.org/) extended this model for RevBayes and also provided [lecture slides online](https://www.slideshare.net/trayc7/integrative-bayesian-analysis-in-revbayes).

## Objective

The goal of this example is to specify a model on an archer's accuracy and precision. We will measure accuracy as `d` = the distance from the bullseye. For our analysis, we will assume that `d ~ Gamma(alpha, beta)`. For this analysis, we will re-paramterize the Gamma so that we focus on the mean and variance of the distribution. 

To compute the mean and variance of a gamma, we simply have to consider:

```
mean = alpha / beta
var = alpha / beta^2
```

## Simulate Data
First we will simulate the the "observed" data under the *true* gamma distribution. This will also allow us to compare our estimated parameters to their true values.

The true accuracy and variance correspond to the true mean and variance of the gamma distribution.

In [1]:
true_accuracy = 35.0
true_variance = 4.0

We compute the true shape (alpha) and rate (beta) of the gamma distribution from the variables above.

In [2]:
true_alpha = (true_accuracy^2) / true_variance
true_beta = true_accuracy / true_variance

Now generate 6 shots (that's how many arrows I have) from the true gamma distribution. This uses the `rgamma()` function which, like the function of the same name in `R` returns a vector of random draws from the gamma.

In [3]:
num_shots = 6
observed_shots = rgamma(num_shots, true_alpha, true_beta)

## Define the Prior Model
Now that we have our observed data entered, we can set up our model

### Stochastic Nodes

In this model we have 2 stochastic nodes, the mean and variance of the gamma distribution. We are calling these nodes `mean` and `var`. 

#### The mean
We are assuming a uniform prior on the mean of the gamma distribution. 

In [4]:
mean ~ dnUnif(10,40)

#### The Variance
We assume the variance comes from a Gamma distribution

In [5]:
var ~ dnGamma(20,2)

### Deterministic Nodes
#### Computing the alpha and beta of the gamma model

Because we are putting priors on the mean and variance, we have to use deterministic nodes to get the shape and rate of the gamma distribution on archery accuracy.

In [6]:
alpha := (mean * mean) / var
beta := mean / var

### The Data
#### Create the node for the data
Now we will create the stochastic node representing the data (d is the distance from the bullseye) for 6 fired shots. To do this, we will put the data `d` in a vector using a `for` loop. Within the loop, we will instantiate `d[i]` for each fire shot as a stochastic node generated by a gamma distribution. Then, we will `clamp()` each node to the corresponding observed shot.

In [7]:
for(i in 1:num_shots){
    d[i] ~ dnGamma(alpha,beta)
    d[i].clamp(observed_shots[i])
}

## The Model Object
Now that our model is fully specified and the observed data are associated with their clamped node, we can create the model object. This is a workspace object that allows us to create a container holding our full graphical model. Within the workspace, we can then use this object (called `mymodel`).

In [8]:
mymodel = model(beta)

### Visualize the Graph
In theory, it would be great to view graphics directly in the notebook. But the graphviz plotting in python does not display the graph in the RevBayes kernel.

In [9]:
mymodel.graph("archery_model.dot")

To use graphviz in python you must first install the graphviz python package: https://pypi.python.org/pypi/graphviz

In [10]:
%%python

import graphviz as gv

f = open('archery_model.dot','r')
ds = f.read()
f.close()
s = gv.Source(ds)
s

Instead, we can view the graphical model if we have graphviz downloaded and set `.dot` files to open with it. To do this, you must install graphviz (http://www.graphviz.org/) and make sure that `*.dot` files open in that program.

In [11]:
%shell open archery_model.dot

### Moves
Now we must add some moves on the stochastic nodes `mean` and `var`.

In [12]:
moves[1] = mvSlide(mean,delta=1.0,tune=true,weight=3.0)
moves[2] = mvScale(var,lambda=1.0,tune=true,weight=3.0)

### Monitors

In [13]:
monitors[1] = mnModel(filename="archery_mcmc_1.log",printgen=10, separator = TAB)
monitors[2] = mnScreen(printgen=1000, mean, var)

### MCMC

In [14]:
mymcmc = mcmc(mymodel, monitors, moves,nruns=1)

### Burnin

In [15]:
mymcmc.burnin(generations=10000,tuningInterval=1000)


   Running burn-in phase of Monte Carlo sampler for 10000 iterations.
   This simulation runs 1 independent replicate.
   The simulator uses 2 different moves in a random move schedule with 6 moves per iteration

Progress:
0---------------25---------------50---------------75--------------100
********************************************************************


### Run

In [16]:
mymcmc.run(generations=30000,underPrior=false)


   Running MCMC simulation
   This simulation runs 1 independent replicate.
   The simulator uses 2 different moves in a random move schedule with 6 moves per iteration

Iter        |      Posterior   |     Likelihood   |          Prior   |           mean   |            var   |    elapsed   |        ETA   |
------------------------------------------------------------------------------------------------------------------------------------------
0           |       -18.8069   |       -13.4655   |       -5.34137   |       36.58019   |       11.08322   |   00:00:00   |   --:--:--   |
1000        |       -19.0876   |       -13.8837   |       -5.20391   |       34.70036   |       8.556915   |   00:00:00   |   --:--:--   |
2000        |       -18.4317   |        -13.284   |       -5.14773   |       36.64006   |        10.1623   |   00:00:00   |   00:00:00   |
3000        |       -18.2059   |       -13.1011   |       -5.10486   |       36.41705   |       9.390706   |   00:00:00   |   00:00:00

## Summarize the data